<a href="https://colab.research.google.com/github/VebZ20/Twitter_Sentiment_Analysis/blob/main/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
pip install numpy pandas scikit-learn tensorflow transformers matplotlib seaborn langchain joblib flask

In [3]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.0 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast
import matplotlib.pyplot as plt
import seaborn as sns
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import joblib
from flask import Flask, request, jsonify

In [ ]:
pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.12
albumentations                   1.4.12
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.2
astropy-iers-data                0.2024.7.29.0.32.7
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                 

In [5]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [6]:
nltk.download(['punkt', 'wordnet',  'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
import csv
import pandas as pd
from io import StringIO

def clean_and_parse_line(line):
    # Remove any leading/trailing whitespace
    line = line.strip()

    # If the line doesn't start and end with quotes, just split by comma
    if not (line.startswith('"') and line.endswith('"')):
        return line.split(',')

    # Remove the starting and ending quotes
    line = line[1:-1]

    # Split the line by '","' to handle internal commas correctly
    parts = line.split('","')

    # If we don't have exactly 6 parts, something is wrong
    if len(parts) != 6:
        # Try to salvage what we can
        return [parts[0]] + parts[1:5] + [','.join(parts[5:])]

    return parts

def read_sentiment140(file_path, num_lines=None):
    data = []
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        for i, line in enumerate(file):
            if num_lines and i >= num_lines:
                break
            try:
                cleaned_line = clean_and_parse_line(line)
                if len(cleaned_line) == 6:
                    data.append(cleaned_line)
            except Exception as e:
                print(f"Error on line {i+1}: {e}")

    return pd.DataFrame(data, columns=["sentiment", "ids", "date", "query", "user", "text"])

# Try to read the entire file
try:
    df = read_sentiment140('training.1600000.processed.noemoticon.csv')
    print("Successfully read the entire file.")
except Exception as e:
    print(f"Error reading the entire file: {e}")

    # If reading the entire file fails, try reading a subset
    try:
        df = read_sentiment140('training.1600000.processed.noemoticon.csv', num_lines=1000000)
        print("Successfully read the first 1,000,000 lines.")
    except Exception as e:
        print(f"Error reading subset of file: {e}")
        df = None

# Check if df is successfully created
if df is not None:
    print(df.head())
    print(df.info())
else:
    print("Failed to read the CSV file.")

Successfully read the entire file.
  sentiment         ids                          date     query  \
0         0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1         0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2         0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3         0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4         0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     No

In [8]:
df

,sentiment,ids,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [9]:
df['sentiment'] = df['sentiment'].str.replace('"', '').astype(int)
print("Unique sentiment values:", df['sentiment'].unique())

df['sentiment'] = df['sentiment'].astype(int)

Unique sentiment values: [0 4]


In [ ]:
df['sentiment']

In [10]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove user @ references and '#' from tweet
    text = re.sub(r'\@\w+|\#\w+', '', text)

    # Remove non-letter characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Remove stopwords and short words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word, pos=get_wordnet_pos(word)) for word in tokens]

    return ' '.join(tokens)

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": nltk.corpus.wordnet.ADJ,
                "N": nltk.corpus.wordnet.NOUN,
                "V": nltk.corpus.wordnet.VERB,
                "R": nltk.corpus.wordnet.ADV}
    return tag_dict.get(tag, nltk.corpus.wordnet.NOUN)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1510532 entries, 0 to 1510531
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1510532 non-null  int64 
 1   ids        1510532 non-null  object
 2   date       1510532 non-null  object
 3   query      1510532 non-null  object
 4   user       1510532 non-null  object
 5   text       1510532 non-null  object
dtypes: int64(1), object(5)
memory usage: 69.1+ MB


In [11]:
df['cleaned_text'] = df['text'].apply(preprocess_text)
df['sentiment'] = df['sentiment'].map({0: 'negative', 4: 'positive'})

In [12]:
# Split the data
X = df['cleaned_text']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [13]:
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (1280000,)
Testing set shape: (320000,)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast
import tensorflow as tf
import joblib
import pickle

# Feature Extraction for traditional ML models
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)



# Label Encoding for traditional ML models
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Feature Selection
print("Performing feature selection...")
selector = SelectFromModel(estimator=RandomForestClassifier(n_estimators=100, random_state=42), max_features=5000)
X_train_selected = selector.fit_transform(X_train_tfidf, y_train_encoded)
X_test_selected = selector.transform(X_test_tfidf)
print(f"Number of features selected: {X_train_selected.shape[1]}")

# Traditional ML Models
models = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(random_state=42),
    'Linear SVC': LinearSVC(random_state=42)
}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train_selected, y_train_encoded)
    y_pred = model.predict(X_test_selected)
    print(f"{name} Accuracy: {accuracy_score(y_test_encoded, y_pred)}")
    print(classification_report(y_test_encoded, y_pred, target_names=le.classes_))

# Random Forest with Randomized Search
print("\nPerforming Randomized Search for Random Forest...")
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_random_search = RandomizedSearchCV(estimator=rf, param_distributions=rf_param_grid,
                                      n_iter=20, cv=3, verbose=2, random_state=42, n_jobs=-1)

rf_random_search.fit(X_train_selected, y_train_encoded)
best_rf_model = rf_random_search.best_estimator_

print("Best Random Forest parameters:", rf_random_search.best_params_)
rf_pred = best_rf_model.predict(X_test_selected)
print(f"Random Forest Accuracy: {accuracy_score(y_test_encoded, rf_pred)}")
print(classification_report(y_test_encoded, rf_pred, target_names=le.classes_))

# Add Random Forest to models dictionary
models['Random Forest'] = best_rf_model

# Gradient Boosting
print("\nPerforming Randomized Search for Gradient Boosting...")
gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

gb = GradientBoostingClassifier(random_state=42)
gb_random_search = RandomizedSearchCV(estimator=gb, param_distributions=gb_param_grid,
                                      n_iter=20, cv=3, verbose=2, random_state=42, n_jobs=-1)

gb_random_search.fit(X_train_selected, y_train_encoded)
best_gb_model = gb_random_search.best_estimator_

print("Best Gradient Boosting parameters:", gb_random_search.best_params_)
gb_pred = best_gb_model.predict(X_test_selected)
print(f"Gradient Boosting Accuracy: {accuracy_score(y_test_encoded, gb_pred)}")
print(classification_report(y_test_encoded, gb_pred, target_names=le.classes_))

# Add Gradient Boosting to models dictionary
models['Gradient Boosting'] = best_gb_model

# LSTM Model
max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# One-hot encode the labels for LSTM
y_train_cat = to_categorical(y_train_encoded)
y_test_cat = to_categorical(y_test_encoded)

lstm_model = Sequential([
    Embedding(max_words, 128, input_length=max_len),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='softmax')  # 2 output neurons for binary classification
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\nTraining LSTM model...")
lstm_history = lstm_model.fit(X_train_pad, y_train_cat,
                              epochs=5, batch_size=64, validation_split=0.1,
                              verbose=1)

lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test_pad, y_test_cat, verbose=0)
print(f"LSTM Test Accuracy: {lstm_accuracy}")

# DistilBERT Model
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
distilbert_model = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize the data
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train_cat
)).shuffle(1000).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test_cat
)).batch(16)

# Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
distilbert_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("\nTraining DistilBERT model...")
distilbert_history = distilbert_model.fit(train_dataset, epochs=3, validation_data=test_dataset)

# Evaluate DistilBERT model
test_loss, test_accuracy = distilbert_model.evaluate(test_dataset)
print(f"DistilBERT Test Accuracy: {test_accuracy}")

# Saving all models and components
print("\nSaving models and components...")

# Save traditional ML models
for name, model in models.items():
    joblib.dump(model, f'{name.replace(" ", "_").lower()}_model.joblib')
    print(f"Saved {name} model")

# Save LSTM model and its tokenizer
lstm_model.save('lstm_model.h5')
with open('lstm_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("Saved LSTM model and tokenizer")

# Save DistilBERT model and tokenizer

distilbert_model.save_pretrained('./distilbert_model')
tokenizer.save_pretrained('./distilbert_tokenizer')
print("Saved DistilBERT model and tokenizer")

# Save preprocessing components
joblib.dump(best_rf_model, 'random_forest_model.joblib')
joblib.dump(best_gb_model, 'gradient_boosting_model.joblib')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')
joblib.dump(le, 'label_encoder.joblib')
joblib.dump(selector, 'feature_selector.joblib')
print("Saved TF-IDF vectorizer, Label Encoder, and Feature Selector")

# Save other important parameters
with open('model_params.pickle', 'wb') as f:
    pickle.dump({
        'max_words': max_words,
        'max_len': max_len
    }, f)
print("Saved model parameters")

print("All models and components have been saved.")

# Function to load models (for future use)
def load_models():
    loaded_models = {}
    for name in models.keys():
        loaded_models[name] = joblib.load(f'{name.replace(" ", "_").lower()}_model.joblib')

    loaded_models['Random Forest'] = joblib.load('random_forest_model.joblib')
    loaded_models['Gradient Boosting'] = joblib.load('gradient_boosting_model.joblib')
    lstm_model = tf.keras.models.load_model('lstm_model.h5')
    with open('lstm_tokenizer.pickle', 'rb') as handle:
        lstm_tokenizer = pickle.load(handle)

    distilbert_model = TFDistilBertForSequenceClassification.from_pretrained('./distilbert_model')
    distilbert_tokenizer = DistilBertTokenizerFast.from_pretrained('./distilbert_tokenizer')

    tfidf_vectorizer = joblib.load('tfidf_vectorizer.joblib')
    label_encoder = joblib.load('label_encoder.joblib')
    selector = joblib.load('feature_selector.joblib')

    with open('model_params.pickle', 'rb') as f:
        model_params = pickle.load(f)

    return loaded_models, lstm_model, lstm_tokenizer, distilbert_model, distilbert_tokenizer, tfidf_vectorizer, label_encoder, model_params, selector

print("\nTo load the models in the future, you can use the load_models() function.")

Performing feature selection...


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast
import tensorflow as tf
import joblib
import pickle

# Feature Extraction for traditional ML models
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
print("Performing feature selection...")
selector = SelectFromModel(estimator=RandomForestClassifier(n_estimators=100, random_state=42), max_features=5000)
X_train_selected = selector.fit_transform(X_train_tfidf, y_train_encoded)
X_test_selected = selector.transform(X_test_tfidf)
print(f"Number of features selected: {X_train_selected.shape[1]}")
# Label Encoding for traditional ML models
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Traditional ML Models
models = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(random_state=42),
    'Linear SVC': LinearSVC(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train_tfidf, y_train_encoded)
    y_pred = model.predict(X_test_tfidf)
    print(f"{name} Accuracy: {accuracy_score(y_test_encoded, y_pred)}")
    print(classification_report(y_test_encoded, y_pred, target_names=le.classes_))


# LSTM Model
max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# One-hot encode the labels for LSTM
y_train_cat = to_categorical(y_train_encoded)
y_test_cat = to_categorical(y_test_encoded)

lstm_model = Sequential([
    Embedding(max_words, 128, input_length=max_len),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='softmax')  # 2 output neurons for binary classification
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\nTraining LSTM model...")
lstm_history = lstm_model.fit(X_train_pad, y_train_cat,
                              epochs=5, batch_size=64, validation_split=0.1,
                              verbose=1)

lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test_pad, y_test_cat, verbose=0)
print(f"LSTM Test Accuracy: {lstm_accuracy}")

# DistilBERT Model
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
distilbert_model = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize the data
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train_cat
)).shuffle(1000).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test_cat
)).batch(16)

# Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
distilbert_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("\nTraining DistilBERT model...")
distilbert_history = distilbert_model.fit(train_dataset, epochs=3, validation_data=test_dataset)

# Evaluate DistilBERT model
test_loss, test_accuracy = distilbert_model.evaluate(test_dataset)
print(f"DistilBERT Test Accuracy: {test_accuracy}")
# Save models
import joblib

for name, model in models.items():
    joblib.dump(model, f'{name.replace(" ", "_").lower()}_model.joblib')

lstm_model.save('lstm_model.h5')
distilbert_model.save_pretrained('./distilbert_model')
tokenizer.save_pretrained('./distilbert_tokenizer')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')
joblib.dump(le, 'label_encoder.joblib')

print("All models have been trained and saved.")

# Saving all models and components
print("\nSaving models and components...")

# Save traditional ML models
for name, model in models.items():
    joblib.dump(model, f'{name.replace(" ", "_").lower()}_model.joblib')
    print(f"Saved {name} model")

# Save LSTM model and its tokenizer
lstm_model.save('lstm_model.h5')
with open('lstm_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("Saved LSTM model and tokenizer")

# Save DistilBERT model and tokenizer
distilbert_model.save_pretrained('./distilbert_model')
tokenizer.save_pretrained('./distilbert_tokenizer')
print("Saved DistilBERT model and tokenizer")

# Save preprocessing components
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')
joblib.dump(le, 'label_encoder.joblib')
print("Saved TF-IDF vectorizer and Label Encoder")

# Save other important parameters
with open('model_params.pickle', 'wb') as f:
    pickle.dump({
        'max_words': max_words,
        'max_len': max_len
    }, f)
print("Saved model parameters")

print("All models and components have been saved.")

# Function to load models (for future use)
def load_models():
    loaded_models = {}
    for name in models.keys():
        loaded_models[name] = joblib.load(f'{name.replace(" ", "_").lower()}_model.joblib')

    lstm_model = tf.keras.models.load_model('lstm_model.h5')
    with open('lstm_tokenizer.pickle', 'rb') as handle:
        lstm_tokenizer = pickle.load(handle)

    distilbert_model = TFDistilBertForSequenceClassification.from_pretrained('./distilbert_model')
    distilbert_tokenizer = DistilBertTokenizerFast.from_pretrained('./distilbert_tokenizer')

    tfidf_vectorizer = joblib.load('tfidf_vectorizer.joblib')
    label_encoder = joblib.load('label_encoder.joblib')

    with open('model_params.pickle', 'rb') as f:
        model_params = pickle.load(f)

    return loaded_models, lstm_model, lstm_tokenizer, distilbert_model, distilbert_tokenizer, tfidf_vectorizer, label_encoder, model_params

print("\nTo load the models in the future, you can use the load_models() function.")